---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [92]:
import pandas as pd
import re

merged = None
reduced = None

def replace_names(name, remove_parens_and_numbers=True):
    replacements = {"Republic of Korea": "South Korea",
                    "United States of America": "United States",
                    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                    "China, Hong Kong Special Administrative Region": "Hong Kong",
                    "Korea, Rep.": "South Korea", 
                    "Iran, Islamic Rep.": "Iran",
                    "Hong Kong SAR, China": "Hong Kong"}
    if name in replacements.keys():
        return replacements[name]
    if remove_parens_and_numbers:
        if ' (' in name:
            return name.split(' (')[0]
        else :
            return ''.join(c if c not in map(str,range(0,10)) else "" for c in name)
    return name

def load_merge_reduce():
    # Load energy DF
    energy = pd.read_excel('Energy Indicators.xls', sheetname='Energy',
        skiprows=17, names=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'],
        parse_cols=[1, 3, 4, 5], skip_footer=38, na_values='...')
    # Convert to GJ
    energy.loc[:, 'Energy Supply'] *= 1e6
    # Replace country names
    energy['Country'] = energy.loc[:, 'Country'].apply(replace_names)
    
    # Load GDP DF and trim to last 10 years
    gdp = pd.read_csv('world_bank.csv', header=4).rename(columns={'Country Name': 'Country'})
    gdp = gdp[list(gdp.columns.values)[:4] + [str(y) for y in range(2006, 2016)]]
    # Replace only listed names
    gdp['Country'] = energy.loc[:, 'Country'].apply(replace_names, args=(False,))
      
    # Load ScimEm DF
    ScimEm = pd.read_excel('scimagojr-3.xlsx')
    
    
    # Merge energy and GDP, and then ScimEm
    merged = pd.merge(energy, gdp, how='inner',on='Country')
    merged = pd.merge(merged, ScimEm, how='inner',on='Country')
    
    # Reduce df to first 15 ranks
    reduced = merged[(merged['Rank'] < 16)][['Country', 'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].set_index('Country')
    return (merged, reduced)

def answer_one():
    merged, reduced = load_merge_reduce()
    return reduced

answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
Australia,14,8831,8725,90765,15606,10.28,107,5.386000e+09,231.0,11.810810,1.021939e+12,1.060340e+12,1.099644e+12,1.119654e+12,1.142251e+12,1.169431e+12,1.211913e+12,1.241484e+12,1.272520e+12,1.301251e+12
Brazil,15,8668,8596,60702,14396,7.00,86,1.214900e+10,59.0,69.648030,4.522426e+09,4.599352e+09,4.619595e+09,4.433354e+09,4.445500e+09,4.477890e+09,4.490036e+09,4.490036e+09,4.498133e+09,4.542669e+09
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.913560e+11,2.012316e+11,2.078570e+11,2.057027e+11,2.175383e+11,2.302417e+11,2.428063e+11,2.524617e+11,2.571989e+11,2.625130e+11
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,2.453369e+11,2.622667e+11,2.715688e+11,2.760539e+11,2.870182e+11,3.059313e+11,3.183030e+11,3.338173e+11,3.484575e+11,3.591967e+11
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,3.087901e+09,3.061633e+09,3.093254e+09,3.050380e+09,3.140509e+09,3.044715e+09,3.094495e+09,3.260221e+09,3.486535e+09,3.605759e+09
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,9.903472e+09,1.112596e+10,1.138342e+10,1.095360e+10,1.163854e+10,1.247902e+10,1.327149e+10,1.372101e+10,1.435537e+10,1.475372e+10
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,6.108881e+10,6.423489e+10,6.555387e+10,6.071351e+10,5.968062e+10,5.951277e+10,5.821097e+10,5.759165e+10,5.738407e+10,5.832781e+10
Iran,13,8896,8819,57470,19125,6.46,72,9.172000e+09,119.0,5.707721,1.364672e+11,1.370483e+11,1.381986e+11,1.291340e+11,1.300938e+11,1.323802e+11,1.301444e+11,1.326038e+11,1.374733e+11,1.415135e+11
Italy,11,10964,10794,111850,26661,10.20,106,6.530000e+09,109.0,33.667230,6.026267e+11,6.408635e+11,6.794031e+11,7.108518e+11,7.550942e+11,8.016818e+11,8.500237e+11,8.972617e+11,9.423391e+11,9.875141e+11


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [93]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [94]:
def answer_two():
    merged, reduced = load_merge_reduce()
    return len(merged.index) - len(reduced.index)

answer_two()

159

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [95]:
def answer_three():
    Top15 = answer_one()
    Top15['avgGDP'] = Top15[[str(y) for y in range(2006, 2016)]].mean(axis=1)
    return Top15['avgGDP'].order(ascending=False)

answer_three()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: order is deprecated, use sort_values(...)


Country
Russian Federation    1.703064e+13
United Kingdom        1.385465e+12
Australia             1.164043e+12
Italy                 7.867660e+11
China                 3.007950e+11
Canada                2.268907e+11
Iran                  1.345057e+11
Spain                 1.001096e+11
India                 6.022990e+10
Germany               1.235856e+10
Japan                 5.221881e+09
Brazil                4.511899e+09
South Korea           4.509228e+09
France                3.192540e+09
United States         1.958698e+08
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [109]:
def answer_four():
    Top15 = answer_one()
    sixth_avg = str(list(answer_three().index)[5])
    return Top15.loc[sixth_avg, '2015'] - Top15.loc[sixth_avg, '2006'] 
    
answer_four()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: order is deprecated, use sort_values(...)


71156992802.113983

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [111]:
def answer_five():
    Top15 = answer_one()
    return Top15.loc[:, 'Energy Supply per Capita'].mean()

answer_five()

157.59999999999999

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [118]:
def answer_six():
    Top15 = answer_one()
    max_percent = Top15.loc[:, '% Renewable'].max()
    return (Top15[Top15['% Renewable'] == max_percent].index.values[0], max_percent)

answer_six()

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [136]:
def answer_seven():
    Top15 = answer_one()
    Top15['Citations Ratio'] = Top15['Self-citations'] / Top15['Citations']
    max_ratio = Top15.loc[:, 'Citations Ratio'].max()
    return (Top15[Top15['Citations Ratio'] == max_ratio].index.values[0], max_ratio)

answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [150]:
def answer_eight():
    Top15 = answer_one()
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    return Top15.sort(columns='Population Estimate', ascending=False).iloc[2].name

answer_eight()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    Top15 = answer_one()
    return "ANSWER"

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    Top15 = answer_one()
    return "ANSWER"

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():
    Top15 = answer_one()
    return "ANSWER"

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    Top15 = answer_one()
    return "ANSWER"

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    Top15 = answer_one()
    return "ANSWER"

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!